# FARA
> Play FARA

- https://playfara.com

Motivation: A large part of playing FARA involves going through certain routines. These routines are fun to figure out, but their novelty wears off as the game progresses. When starting a new game of FARA, it is preferable to thoroughly search, harvest, and craft in the initial locality. It would be nice to automate away this early game routine. Another such routine is preparing for specific journeys which could involve harvesting and preparing shelter, meals, and rest.

- [ ] **Available Moves**: Learn what the available moves are by parsing the gamelog. And maybe in which situations they are useful.
- [ ] **Satchel**: Learn to craft a burlap satchel in the first area.

Other ideas that could be fun:

- draw a map from the gamelog
- recuperate

Boring ideas that could be done:

- farming :-1:

## Available Moves

- Given a game log ending with `> `, output valid commands
- Probably want to start with a manually played game log. This would bootstrap the available commands. The game should be played without a map, so the player is forced to use the `/` commands only.
- Labeling. As long as there is a valid, non-empty result, we can assume that the label is positive. When the wrong action is taken, we can say it's negative.
- We might need to know how to determine that the game ended, etc.. But we probably don't need to do anything too sophisticated. We could just cap the log at several thousand attempts to take a turn.
- Remember, that we're going to be generating the game logs in one phase, but training in a separate phase. Even when we get around to self play it should be separated.

## Satchel

- The actual recipe is fairly constant. But it may be worth it to learn the recipe through a model. There is some need to understand English in order to discover the ingredients. For example, understanding "2 Rolls of Burlap" vs. "Roll of Burlap".
- Could a bot say which ingredients are necessary given a target item?
- The initial map and its resource location is random.

If coding the algorithm, it'd be something like this:
1. Identify ingredients to gather by recursively calling `/craft`.
2. Find and Harvest the ingredients.
3. Craft the ingredients.

## Starting FARA 

Initialize the Selenium driver.
Unless you use headless mode, you'll want to keep this new Firefox window open as long as you are playing.

At this point, we will go through the steps necessary to start a game. And do so in a way that the game log can be saved. We'll use the log later to generate training data.

Time to get it saving somewhere more manageable. I'll try to overwrite the same file for now though.

In [2]:
! rm -f /tmp/fara/game/*
! ls /tmp/fara/game/


In [3]:
import os

from selenium import webdriver

# TODO: patch to a class
def set_logging_dir(p):
  log_dir = p
  ! mkdir -p $p
  ! rm $p/*
  fp = webdriver.FirefoxProfile()

  fp.set_preference("browser.download.folderList",2)
  fp.set_preference("browser.download.manager.showWhenStarting",False)
  fp.set_preference("browser.download.dir", p)
  # FIXME: these aren't working.
  # fp.set_preference("browser.helperApps.neverAsk.saveToDisk", "text/plain")
  fp.set_preference("browser.helperApps.alwaysAsk.force", False)

  return (fp, p)

In [76]:
from selenium.webdriver.firefox.options import Options
import time

# probably best to maintain just a single global driver to start.
# TODO: use nbdev and go around all this global nonesense with a class.
DRIVER = None
def start_game():
  global DRIVER
  if DRIVER:
    try: DRIVER.quit()
    except: pass

  options = Options()
  # options.headless = True
  fp, log_dir = set_logging_dir("/tmp/fara/game")
  DRIVER = webdriver.Firefox(options=options,
    firefox_profile=fp
  )
  time.sleep(1)
  DRIVER.get("https://brianiscreative.itch.io/fara")
  time.sleep(1)
  # TODO: Wait until it's loaded
  from selenium.webdriver.common.by import By
  DRIVER.find_element_by_class_name("load_iframe_btn").click()
  time.sleep(1)
  # TODO: Wait? until it's loaded
  g = DRIVER.find_element(by="tag name", value="body")
  [g.send_keys(c+"\n") for c in ["/setname bot", "/setclass soldier"]]
  time.sleep(1)
  [g.send_keys(c+"\n") for c in ["/1", "/setdesc", "/stopsharing", "/hd", "/showinput", "/logging"]]
  
  # Since the neverAsk option isn't working, we'll need to force the save via the `/savelog` command.
  # The user will need to click "Save File" and "Do this automatically for files like this from now on."
  g.send_keys("/savelog\n")
  input("""
    Since the neverAsk option isn't working, we'll need to force the save via the `/savelog` command.
    The user will need to click "Save File" and "Do this automatically for files like this from now on."
    Hit enter in this prompt when this is done.
  """)
  
  return g, log_dir


In [59]:
%%time
g, log_dir = start_game()

rm: /tmp/fara/game/*: No such file or directory



    Since the neverAsk option isn't working, we'll need to force the save via the `/savelog` command.
    The user will need to click "Save File" and "Do this automatically for files like this from now on."
    Hit enter in this prompt when this is done.
   


CPU times: user 27.7 ms, sys: 18.7 ms, total: 46.5 ms
Wall time: 20.7 s


The log is now ready to be saved. 

If it was set up by the expectations above, the log should be output to a file that should starts with `The Life and Times of bot the Soldier` and ends with `.txt`.

For now the gamelog is cumulative, but it will be possible [to clear it at some point](https://discord.com/channels/448497182392451073/448498300723789845/814283733988933652). 

Also at some point, we can try to autosave the file with this FAQ: https://selenium-python.readthedocs.io/faq.html?highlight=file#how-to-auto-save-files-using-custom-firefox-profile

In [66]:
!cat /tmp/fara/game/*

Gaming logging  ENABLED
> /savelog


## Making Moves

Let's set up a short function to make a move and output the game log after each move.

In [107]:
import time

def move(m, g=None, debug=False, savelog=True):
  if not g:
    g = DRIVER.find_element(by="tag name", value="body")
  # [g.send_keys(x + "\n") for x in [m, "/savelog"]]
  g.send_keys(f"{m}\n")
  if savelog:
    g.send_keys("/savelog\n")
    # FIXME: would be nice to not need to avoid this,
    # but once every ten turns the save seems to be too slow.
    time.sleep(.1)
    ! mv '{log_dir + "/The Life and Times of bot the Soldier.txt"}' '{log_dir + "/log"}'
  if debug:
    ! ls -latr {log_dir}
    ! tail {log_dir}/log
  

In [64]:
! echo $log_dir

/tmp/fara/game


In [ ]:
move("/lt", g, debug=True)

There are a lot of moves that can be made.
We are interested in training the bot to work in exploration mode. Not the travel mode.

Let's list a few available moves.

In [109]:
MOVES = ["/" + item for sublist in
          [
            ["move " + d for ds in ["n ne e se s sw w nw".split(" ")] for d in ds],
            ["look", "harvest"],
            # Can't do commands like "use", since they prompt for input.
            # No point in using commands like "recipes" since they don't log.
            ["approach " + str(i) for i in range(1,11)],
            [x + " 1" for x in ["target", "take", "harvest"]],
            ["craft " + x for x in ["burlap satchel", "roll of burlap"]]
          ] for item in sublist
        ]
print(MOVES)

['/move n', '/move ne', '/move e', '/move se', '/move s', '/move sw', '/move w', '/move nw', '/look', '/harvest', '/approach 1', '/approach 2', '/approach 3', '/approach 4', '/approach 5', '/approach 6', '/approach 7', '/approach 8', '/approach 9', '/approach 10', '/target 1', '/take 1', '/harvest 1', '/craft burlap satchel', '/craft roll of burlap']


That should make for a nice format to store available moves in for debugging. Might not need it though.

Let's move on to parsing the game log.

## The Gamelog

### Persisting it

First we'll want to copy it over somewhere more persistent than the temp directory.

In [68]:
def persist_log(src=None, tgt=None):
  if not src:
    src = f"{log_dir}/log"
  if not tgt:
    ts = ! date +%s
    ts=ts[0]
    tgt = f"fara_logs/{ts}"
  !cp {src} {tgt}
  print(tgt)

In [13]:
# ! rm fara_logs/*

In [14]:
ts = ! date +%s
ts=ts[0]
persist_log(f"{log_dir}/log", f"fara_logs/{ts}")
!tail fara_logs/{ts}

fara_logs/1614639855
Gaming logging  ENABLED
> /savelog
> /lt
You see an Adventurer  nine steps to the southeast,  a Shrouded Figure you are targeting  nine steps to the southeast,  and  an Orange Tree  nine steps to the north.
> /savelog
> /lt
You see an Adventurer  nine steps to the southeast,  a Shrouded Figure you are targeting  nine steps to the southeast,  and  an Orange Tree  nine steps to the north.
> /savelog


Now that it's persisted, let's extract all the commands from the saved log. 

### Parsing it

Let's not lose track of the goal here:

> Given a game log ending with `> `, output the next command.

I think this parsing is just a way to build up a custom vocabulary.

In [69]:
def extract_inputs(p=None):
  if not p:
    p = !ls fara_logs/ | tail -n 1
    p = f"fara_logs/{p[0]}"
  lines = !cat {p}
  inputs = [line[2:] for i, line in enumerate(lines)
            if line[0:2] == "> " and line not in ["> " + bl for bl in ["/savelog", "/hd", "/help"]]
            and (not lines[i+1].endswith("is not a valid command."))
           ]
  return inputs

In [16]:
# !grep 'valid' fara_logs/`ls fara_logs/ | tail -n 1`

In [17]:
print(set(extract_inputs()))

{'/lt'}


Next we could try to build a labeled data set. But we should probably hit the breaks, step back, and build a random bot first. Once we have something that can make valid but random moves, we can try to substitute the random bot for a trained both. We should be able to re-use the skeleton from the random bot.

## Random Strategy

Soon we can move on to predicting the next move. But for now, let's create something that can generated a game log from a hard-coded list of inputs.

To start, our inputs can come from our latest game log.

In [115]:
import random

def _moves(moves, moves_path = None):
  if not moves:
    moves = list(set(extract_inputs(moves_path)))
  if not moves:
    moves = MOVES
  return moves

def game(strategy='random', moves = [], moves_path=None, final_turn = 3):
  moves = _moves(moves)
  g, log_dir = start_game()
  for t in range(final_turn):
    if strategy == 'random':
      m = random.choice(moves)
      # print(m)
      # when moving randomly we can afford to save only once in a while.
      move(m, debug=False, savelog=(i%10 == 0))
  persist_log()
  return str(moves), g, log_dir

In [111]:
# game(moves_path='fara_logs/1614638272')
# g.quit()

WebDriverException: Message: Failed to decode response from marionette


Okay, that plays nicely.

I am seeing issues with the DRIVER not quiting the previous round, but we can address that later.

Let's see what a longer game looks like.

In [114]:
_, g, _ = game(moves=MOVES, final_turn=100)


    Since the neverAsk option isn't working, we'll need to force the save via the `/savelog` command.
    The user will need to click "Save File" and "Do this automatically for files like this from now on."
    Hit enter in this prompt when this is done.
   


fara_logs/1614649593


Watching that is fine. Let's keep playing though

In [138]:
def play_longer(g, turns, moves = [], strategy='random', tgt=None):
  moves = _moves(moves)
  for t in range(turns):
    if strategy == 'random':
      m = random.choice(moves)
      # print(m)
      # when moving randomly we can afford to save only once in a while.
      move(m, debug=False, savelog=(t%100 == 0 or t==turns-1))
  persist_log(tgt=tgt)

In [140]:
play_longer(g, 1111, moves=MOVES, tgt="fara_logs/latest")

mv: /tmp/fara/game/The Life and Times of bot the Soldier.txt: No such file or directory
fara_logs/latest


In [153]:
!ls -hl fara_logs/

total 1920
-rw-r--r--@ 1 ewolfson  staff   479K Mar  1 22:51 1614657062
-rw-r--r--@ 1 ewolfson  staff   479K Mar  1 21:51 latest


In [143]:
! cp fara_logs/latest fara_logs/`date +%s`

## Labeling data

We have a fairly large log available. Let's use it to write some code for labeling the data.  It's only making a few moves, but it has enough variety in it to use for labelling.

The content of the game log can act as its own label, similar to an LSTM model. The words `You cannot` or `/` following a command means that the command was invalid. A "negative" label. If we pass the model a command (and at least some of the preceeding game log), we can train the model to predict whether the command is valid. If the model can predict that it cannot do a given command, it might know enough about the game to know which moves it can do.

We still have the problem of bootstrapping commands, but that list is enumerable, even manually. And we can take shortcuts to bootstrap it initially. Similar to how the `MOVES` list is basically hardcoded.

We might not want to train the model to predict each word in the log. Though that is the common approach for LSTM and many text predictors.

Remember, as always the final goal for this exercise.

> Given a game log ending with `> `, output the next command.

Here are some basic statistics on the different moves taken.

In [152]:
! grep '> ' fara_logs/latest | grep -v '> /savelog' | sort | uniq -c
! grep '> ' fara_logs/latest | grep -v '> /savelog' | wc -l

 163 > /approach 1
 172 > /approach 10
 184 > /approach 2
 156 > /approach 3
 147 > /approach 4
 171 > /approach 5
 149 > /approach 6
 148 > /approach 7
 162 > /approach 8
 163 > /approach 9
   1 > /arstar
 153 > /craft burlap satchel
 163 > /craft roll of burlap
 184 > /harvest
 174 > /harvest 1
 173 > /look
 161 > /move e
 180 > /move n
 174 > /move ne
 148 > /move nw
 166 > /move s
 145 > /move se
 166 > /move sw
 170 > /move w
 166 > /take 1
 167 > /target 1
    4106


Some moves are always valid, such as `/move ..` or `/look`. While others are only valid when the target is amenable to that action. `/harvest` only works on something harvestable. And finally, there are commands that are always invalid (such as `/arstar`).

At this point it may make sense to bring in ~the big guns~ the fast.ai library to make sure we're processing the data for training in a way that works with their DataLoaders, etc..

## Interface via Selenium

:warning: this is just cruft that came from `cyber-wanderer` branch. Could be useful at some point.

In [ ]:
from selenium.webdriver.common.keys import Keys
import random
import re
import io
import sys

lm = [
    Keys.ARROW_DOWN, Keys.ARROW_LEFT, Keys.ARROW_UP, Keys.ARROW_RIGHT,
    '.', 'x'
]

#################
# Random Strategy
# effectively, causes staff to be used less often
# longer games, but not necessarily more points.
costs = [1/cost for cost in [1,1,1,1,1,10]]
def random_move(b):
    return random.choices(lm, costs)
#################

#################
# Manual Strategy [WIP]
def human_move(b):
    return sys.stdin.read(1)
#################

def board(g):
    # .text is a little too magical (writes \n instead of <br>, etc..)
    return g.find_element(by="id", value="tsv").get_attribute('innerHTML')

def game_score(g):
    return None
#     # .text is easier to regex than innerHTML
#     game_over = re.search("Your final score is (\d+) after ", g.text)
#     # scoring function is the score, but alternative scoring
#     # functions could count kills, visits, keys, etc..
#     return game_over.group(1) if game_over else None

def turn(g):
    s = game_score(g)
    b = board(g)
    # How we get m will by the strategy
    m = random_move(b)
    # m = human_move(board(g))
    if not s: g.send_keys(m)
    return (s, f"{b}\t{m}")

def turn_score(b, g):
    return 0
#     try:
#         return int(re.match("Self: (?:\<[^>]+\>)*(\w+)", b.split("\t")[2500]).group(1))
#     except ValueError:
#         return int(re.search("Your final score is (\d+) after ", g.text).group(1))

def reward(b1, b2, g):
    return turn_score(b2, g) - turn_score(b1, g)

# def game(driver, save_as=None):
#     driver.get("https://brianiscreative.itch.io/fara")
#     g = driver.find_element(by="tag name", value="body")
#     [g.send_keys(c) for c in ["/stopsharing", "/hd", "/showinput", "/logging"]]
#     score = None
#     history = []
#     while score is None:
#         score, b = turn(g)
#         r = reward(b, board(g), g)
#         # make sure not to leak the moves and rewards in training
#         history.append(f"{b}\t{r}")
#     # print(history)
#     if save_as:
#         with open(f"{save_as}", "w") as f: f.write("\n".join(history))
#     return int(score), len(history)